In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [9]:
data = loadmat('ex3data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [10]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

In [11]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [12]:
def cost(theta, X, y, alpha):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1-y), np.log(1 - sigmoid(X * theta.T)))
    reg = (alpha/(2*len(X))) * np.sum(np.power(theta[:,1:theta.shape[1]],2))
    return np.sum(first - second) / len(X) + reg

In [21]:
def gradient(theta, X, y, alpha):
    theta = np.matrix(X)
    y = np.matrix(y)
    
    paramaters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y
    grad = ((X.T * error) / len(X)).T + ((alpha / len(X)) * theta)
    
    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()

In [29]:
from scipy.optimize import minimize
def one_vs_all(X, y, num_labels, alpha):
    rows = X.shape[0]
    params = X.shape[1]
    
    # [k, n+1] array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    
    #insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis = 1)
    
    #lables are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))
        
        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, alpha), method='TNC', jac=gradient)
        all_theta[i-1, :] = fmin.x
        #import scipy.optimize as opt
        #theta,_,_ = opt.fmin_tnc(func=cost, x0=theta, fprime=gradient, args=(X, y, alpha))
        #print(theta)
        
    return all_theta

In [33]:
rows = data['X'].shape[0]
dims = data['X'].shape[1]

all_theta = np.zeros((10, dims + 1))
X = np.insert(data['X'], 0, values = np.ones(rows), axis = 1)
theta = np.zeros(dims + 1)
theta = theta.reshape(theta.shape[0], 1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401, 1), (10, 401))

In [35]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [37]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

ValueError: tnc: invalid gradient vector from minimized function.